# 🌍 Exploratory Data Analysis - World Happiness Report (2015–2019)


## Pendahuluan

Dataset ini berisi laporan kebahagiaan dunia (World Happiness Report) dari tahun 2015 hingga 2019. 
Setiap baris mewakili satu negara pada satu tahun tertentu, dengan beberapa fitur yang berkontribusi pada skor kebahagiaan (Happiness Score).

**Tujuan EDA:**
- Membersihkan data (handling missing values & duplicates)
- Melakukan eksplorasi data deskriptif
- Menyiapkan data untuk dashboard interaktif Streamlit


1. Import dan Gabungkan Data

In [20]:
# Import Library
import pandas as pd
import os

# Lokasi file data tahunan (dalam folder 'data')
data_path = "data"
files = sorted([f for f in os.listdir(data_path) if f.endswith(".csv")])
print("File yang ditemukan:", files)

# Mapping nama kolom agar seragam dari berbagai tahun
column_mapping = {
    'Happiness.Rank': 'Happiness Rank',
    'Overall rank': 'Happiness Rank',
    'Happiness.Score': 'Happiness Score',
    'Score': 'Happiness Score',
    'Country or region': 'Country',
    'Country': 'Country',
    'Economy (GDP per Capita)': 'GDP per capita',
    'Economy..GDP.per.Capita.': 'GDP per capita',
    'GDP per capita': 'GDP per capita',
    'Health (Life Expectancy)': 'Healthy life expectancy',
    'Health..Life.Expectancy.': 'Healthy life expectancy',
    'Healthy life expectancy': 'Healthy life expectancy',
    'Family': 'Social support',
    'Social support': 'Social support',
    'Freedom': 'Freedom to make life choices',
    'Freedom to make life choices': 'Freedom to make life choices',
    'Trust (Government Corruption)': 'Perceptions of corruption',
    'Trust..Government.Corruption.': 'Perceptions of corruption',
    'Perceptions of corruption': 'Perceptions of corruption',
    'Generosity': 'Generosity',
    'Dystopia.Residual': 'Dystopia Residual',
    'Dystopia Residual': 'Dystopia Residual',
}

# Inisialisasi list kosong
all_dfs = []

for file in files:
    year = int(file.replace(".csv", ""))
    df = pd.read_csv(os.path.join(data_path, file))

    # Rename kolom
    df.rename(columns=column_mapping, inplace=True)
    df["Year"] = year
    all_dfs.append(df)

# Gabungkan semua data
df_all = pd.concat(all_dfs, ignore_index=True)
df_all = df_all.loc[:, ~df_all.columns.duplicated()]  # Hapus kolom duplikat
print("Ukuran data gabungan:", df_all.shape)


File yang ditemukan: ['2015.csv', '2016.csv', '2017.csv', '2018.csv', '2019.csv']
Ukuran data gabungan: (782, 17)


2. Pemeriksaan Awal dan Cleaning

In [21]:
# Pilih hanya kolom yang dibutuhkan untuk dashboard
main_columns = [
    'Country', 'Year', 'Happiness Score', 'GDP per capita',
    'Social support', 'Healthy life expectancy',
    'Freedom to make life choices', 'Generosity',
    'Perceptions of corruption', 'Dystopia Residual'
]

# Ambil kolom yang tersedia
selected_columns = [col for col in main_columns if col in df_all.columns]
df_clean = df_all[selected_columns]

# Cek Missing Values awal
print("Jumlah missing value sebelum pembersihan:\n", df_clean.isnull().sum())

# Drop baris jika kolom krusial kosong (Country dan Happiness Score)
df_clean = df_clean.dropna(subset=['Country', 'Happiness Score'])

# Daftar faktor utama
factors = [
    'GDP per capita', 'Social support', 'Healthy life expectancy',
    'Freedom to make life choices', 'Generosity',
    'Perceptions of corruption', 'Dystopia Residual'
]

# Drop baris jika SEMUA nilai di faktor kosong (tapi biarkan jika hanya sebagian kosong)
df_clean = df_clean.dropna(how='all', subset=factors)

# Ganti nilai NaN yang tersisa pada faktor dengan 0 agar visualisasi aman (khusus pie chart)
df_clean[factors] = df_clean[factors].fillna(0)

# Drop duplikat
print("Jumlah duplikat sebelum dihapus:", df_clean.duplicated().sum())
df_clean = df_clean.drop_duplicates()

# Cek ulang
print("Ukuran data setelah pembersihan:", df_clean.shape)
print("Jumlah missing value setelah pembersihan:\n", df_clean.isnull().sum())


Jumlah missing value sebelum pembersihan:
 Country                           0
Year                              0
Happiness Score                   0
GDP per capita                    0
Social support                    0
Healthy life expectancy           0
Freedom to make life choices      0
Generosity                        0
Perceptions of corruption         1
Dystopia Residual               312
dtype: int64
Jumlah duplikat sebelum dihapus: 0
Ukuran data setelah pembersihan: (782, 10)
Jumlah missing value setelah pembersihan:
 Country                         0
Year                            0
Happiness Score                 0
GDP per capita                  0
Social support                  0
Healthy life expectancy         0
Freedom to make life choices    0
Generosity                      0
Perceptions of corruption       0
Dystopia Residual               0
dtype: int64


3. Simpan Versi Bersih

In [22]:
# Simpan ke CSV
df_clean.to_csv("world_happiness_clean.csv", index=False)
print("Dataset bersih disimpan sebagai 'world_happiness_clean.csv'")

Dataset bersih disimpan sebagai 'world_happiness_clean.csv'
